In [20]:
from googleapiclient.discovery import build
from google.auth import default
from google.colab import auth

# Step 1: Authenticate and initialize Google Docs API
auth.authenticate_user()
creds, _ = default()
docs_service = build('docs', 'v1', credentials=creds)

# Step 2: Create a new Google Document
document = docs_service.documents().create(body={"title": "Markdown to Google Docs"}).execute()
document_id = document.get('documentId')
print(f"Document created successfully with ID: {document_id}")

# Step 3: Read Markdown file content
def read_md_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Step 4: Prepare Markdown content for Google Docs
def format_md_for_google_docs(md_content):
    lines = md_content.split('\n')
    requests = []

    # Current insertion index
    index = 1

    # Initialize listId variables for bulleted and checkbox lists
    list_id_bullet = None
    list_id_checkbox = None

    for line in lines:
        # Heading 1 (Main Title)
        if line.startswith('# '):
            requests.append({
                "insertText": {
                    "text": line[2:] + "\n",
                    "location": {"index": index}
                }
            })
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index, "endIndex": index + len(line[2:]) + 1},
                    "paragraphStyle": {"namedStyleType": "HEADING_1"},
                    "fields": "namedStyleType"
                }
            })
            index += len(line[2:]) + 1

        # Heading 2 (Section Headers)
        elif line.startswith('## '):
            requests.append({
                "insertText": {
                    "text": line[3:] + "\n",
                    "location": {"index": index}
                }
            })
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index, "endIndex": index + len(line[3:]) + 1},
                    "paragraphStyle": {"namedStyleType": "HEADING_2"},
                    "fields": "namedStyleType"
                }
            })
            index += len(line[3:]) + 1

        # Heading 3 (Sub-section Headers)
        elif line.startswith('### '):
            requests.append({
                "insertText": {
                    "text": line[4:] + "\n",
                    "location": {"index": index}
                }
            })
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index, "endIndex": index + len(line[4:]) + 1},
                    "paragraphStyle": {"namedStyleType": "HEADING_3"},
                    "fields": "namedStyleType"
                }
            })
            index += len(line[4:]) + 1


        # Checkboxes
        elif line.strip().startswith('- [ ]'):
            content = line.replace('- [ ]', '').strip()

            requests.append({
                "insertText": {
                    "text": content + "\n",
                    "location": {"index": index}
                }
            })

            # Create a checkbox list item (using CreateParagraphBulletsRequest)
            requests.append({
                "createParagraphBullets": {
                    "range": {"startIndex": index, "endIndex": index + len(content) + 1},
                    "bulletPreset": "BULLET_CHECKBOX"
                }
            })

            requests.append({
              "updateTextStyle": {
                  "range": {"startIndex": index  , "endIndex": index + len(content) + 1},
                  "textStyle": {
                      "foregroundColor": {
                          "color": {
                              "rgbColor": {
                                  "red": 165 / 255,  # Normalizing the RGB values to 0-1
                                  "green": 42 / 255,
                                  "blue": 42 / 255
                              }
                          }
                      }
                  },
                  "fields": "foregroundColor"
              }
            })

            index += len(content) + 1

        # Primary Bullet Points (Circle)
        elif line.strip().startswith('- '):
            content = line.strip()[2:]

            requests.append({
                "insertText": {
                    "text": content + "\n",
                    "location": {"index": index}
                }
            })

            # Create a bullet list item (this uses CreateParagraphBulletsRequest)
            requests.append({
                "createParagraphBullets": {
                    "range": {"startIndex": index, "endIndex": index + len(content) + 1},
                    "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                }
            })

            index += len(content) + 1

        elif line.strip().startswith('*'):
            content = line.strip()[2:]
            indentation_level = len(line) - len(line.lstrip())
            nesting_level = (indentation_level // 2) + 1  # Ensure it's at least second level

            # Format the content with spaces for indentation
            text_with_tabs = "\t"*nesting_level + "▪ "+content

            # Step 1: Insert the text with proper indentation
            requests.append({
                "insertText": {
                    "text": text_with_tabs + "\n",
                    "location": {"index": index}
                }
            })
            index += len(text_with_tabs) + 1
        # Footer (distinct style)
        elif "Meeting recorded by" in line or "Duration" in line:
            requests.append({
                "insertText": {
                    "text": line + "\n",
                    "location": {"index": index}
                }
            })
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": index, "endIndex": index + len(line) + 1},
                    "textStyle": {"italic": True},
                    "fields": "italic"
                }
            })
            index += len(line) + 1

        # Regular text
        else:
            requests.append({
                "insertText": {
                    "text": line + "\n",
                    "location": {"index": index}
                }
            })
            index += len(line) + 1

    return {"requests": requests}

# Step 5: File Path for Markdown
file_path = 'contents.md'
md_content = read_md_file(file_path)

# Step 6: Format the Markdown for Google Docs
formatted_content = format_md_for_google_docs(md_content)

# Step 7: Perform the batch update on the document
try:
    batch_update_response = docs_service.documents().batchUpdate(
        documentId=document_id, body=formatted_content).execute()
    document_url = f"https://docs.google.com/document/d/{document_id}/edit"
    print(f'Document Link: {document_url}')
    print("Content successfully uploaded to Google Docs.")
except Exception as e:
    print(f"An error occurred: {e}")


Document created successfully with ID: 1OKDWX2fpeAP53VBK8AQf4rX5dNi1C7nKXUi_WKIdrvE
Document Link: https://docs.google.com/document/d/1OKDWX2fpeAP53VBK8AQf4rX5dNi1C7nKXUi_WKIdrvE/edit
Content successfully uploaded to Google Docs.
